In [113]:
#READ ME
#This code is written by utilizing the following resources:
#1)https://www.keysight.com/us/en/library/manuals/help-file/e5061b-network-analyzer-pdf-help-for-firmware-rev-a020x-1842257
#2)https://docs.keysight.com/kkbopen/e5063a-python-example-for-basic-measurement-setup-and-calibration-607279081.html
#3)https://docs.keysight.com/kkbopen/python-example-program-for-e5080b-csv-trace-data-save-and-binary-transfer-to-controller-pc-721232696.html

In [35]:
from tkinter import *
from tkinter import ttk #This interface allow us to draw windows
import time
import pyvisa

In [36]:
rm = pyvisa.ResourceManager()
# Input the target instrument IP address
Addr = 'GPIB0::17::INSTR'
inst = rm.open_resource(Addr)
root = Tk()
#    return inst

## Identify the Instrument

In [37]:
def connection_control():
    inst.write('*RST')
    inst.write('*CLS')
    # Query Instrument ID
    idn = inst.query('*IDN?')
    frame = ttk.Frame(root, padding=100)
    frame.grid()
    lab = ttk.Label(frame, text = 'Connection to '+idn+'was succesful',justify='left')
    lab.grid(column=0, row=0)
    root.mainloop()

In [38]:
connection_control()

## Wait for Instrument to Finish Procedure and Preset the Instrument

In [6]:
#when you write a program that issues multiple calibration commands in series, you should use the *OPC?
# command or some other means to ensure that no command is executed before the preceding command completes itself.
def wait():
    inst.write('*OPC')
    sta = 0
    while (sta & 1 ) == 0:
        sta = int(inst.query('*ESR?'))
        #This command reads the value of the Standard Event Status Register. Execution of this command clears the register value.
        time.sleep(1)
    return sta

# Preset the instrument
def Preset():
    inst.write(':SYST:PRES')

In [44]:
Preset()

## Set the Frequency, Sweep Points, and IF Bandwidth

In [51]:
def set_frequency(channel_no, start, stop, points,autoIF, IF = 10):
    inst.write(':SENS%d:FREQ:STAR %s' %(int(channel_no), str(start)))
    inst.write(':SENS%d:FREQ:STOP %s' %(int(channel_no), str(stop)))
    inst.write(':SENS%d:SWE:POIN %d' %(int(channel_no), int(points)))
    if autoIF == 1:
        inst.write(':SENS%d:BWA ON' %(int(channel_no)))
    else:
        inst.write(':SENS%d:BAND:RES %s' %(int(channel_no), str(IF)))

In [52]:
set_frequency(channel_no=1,start=100, stop=10e3, points=1000,autoIF=1)

## Set Number of Traces

In [10]:
# Set numbers of traces
def set_no_traces(channel_no, NTrace):
    inst.write(':CALC%d:PAR:COUN %s'%(int(channel_no), str(NTrace)))

In [53]:
set_no_traces(1,3)

## Set S-parameters

In [31]:
# Set S-Parameters such as S11, S12, S21, S22
def set_s_parameters(channel_no, trace, s_parameters):
    inst.write(':CALC%d:PAR%d:DEF %s' %(int(channel_no), int(trace), str(s_parameters)))

set_s_parameters(1,1,"S12")

# Set Impedance

In [61]:
def set_impedance(channel_no,trace, mode = "Z"):

    inst.write(':CALC%d:PAR%d:DEF %s' %(int(channel_no), int(trace), str(mode)))

set_impedance(1,2,mode="S22")

## Choose Trace Format

In [118]:
# Choose trace format
#def trace_format(a,trace,m1,m2):
#    if a in m1:
#        ind = m1.index(a)
#        inst.write(':CALC1:TRAC%d:FORM %s' %(trace,m2[ind]))
def trace_format(channel_no, form):
    inst.write(':CALC%d:FORM %s' %(int(channel_no), str(form)))

## Check if Command is Completed

In [119]:
def check_status(windows,state):
    if int(state) == 1:
        lab= ttk.Label(windows, text = 'Procedure Completed')
        lab.place(relx=.1, rely=.85)
    else:
        lab= ttk.Label(windows, text = inst.query(':SYST:ERR?'))
        lab.place(relx=.1, rely=.85)

## Save Trace to File (.csv file)

In [120]:
def save_trace_csv(trace_name):
    inst.write(':MMEM:STOR:FDAT %s.csv' % str(trace_name))
    state = wait()
    if int(state) == 1:
        lab= ttk.Label(root, text = 'Trace Saved')
        lab.place(relx=.02, rely=.7)

## Save Measurement Data in Touchstone Format

In [121]:
def save_trace_s2p(data_type, trace_name):
    #To set the data type of the files saved in touchstone format, use the following command:
    inst.write(':MMEM:STOR:SNP:FORM %s' % str(data_type)) #Data types:{DB: decibel-angle  RI: real-imaginary MA: magnitude-angle}
    #To determine the file type in touchstone file format and specify a port, use one of the following commands according to the number of ports used:
    inst.write(':MMEM:STOR:SNP:TYPE:S2P 1,2') #port 1 and port 2
    #To save measurement data in touchstone format, use the following command:
    inst.write(':MMEM:STOR:SNP %s' % str(trace_name))
    state = wait()
    if int(state) == 1:
        lab= ttk.Label(root, text = 'Trace Saved')
        lab.place(relx=.02, rely=.7)

def save_trace_s1p(data_type, port_name, trace_name):
    #To set the data type of the files saved in touchstone format, use the following command:
    inst.write(':MMEM:STOR:SNP:FORM %s' % str(data_type)) #Data types:{DB: decibel-angle  RI: real-imaginary MA: magnitude-angle}
    #To determine the file type in touchstone file format and specify a port, use one of the following commands according to the number of ports used:
    inst.write(':MMEM:STOR:SNP:TYPE:S1P %s' % str(port_name)) #port 1 or port 2
    #To save measurement data in touchstone format, use the following command:
    inst.write(':MMEM:STOR:SNP %s' % str(trace_name))
    state = wait()
    if int(state) == 1:
        lab= ttk.Label(root, text = 'Trace Saved')
        lab.place(relx=.02, rely=.7)

## 2 - port calibration

In [ ]:
def calibration_1_port(channel_no, port_no):
    #Syntax= :SENSe{[1]-4}:CORRection:COLLect:METHod:SOLT1 <numeric>
    #Description= This command sets the calibration type to the 1-port calibration of the specified port, for the selected channel (Ch).
    inst.write(':SENS%d:CORR:COLL:METH:SOLT1 %d' %(int(channel_no), int(port_no)))

def calibration_2_port(channel_no, port_no_1, port_no_2):
    #Syntax= :SENSe{[1]-4}:CORRection:COLLect:METHod:SOLT2 <numeric 1>,<numeric 2>
    #Description= This command sets the calibration type to the full 2-port calibration between the specified 2 ports, for the selected channel (Ch).
    inst.write(':SENS%d:CORR:COLL:METH:SOLT2 %d,%d' %(int(channel_no), int(port_no_1), int(port_no_2)))

#To select a calibration kit
def select_calibration_kit(channel_no, calkit_name):
    inst.write(':SENS%d:CORR:COLL:CKIT %s' %(int(channel_no), str(calkit_name)))

def check_calibration_type(channel_no, trace_no):
    cal_type = inst.query(':SENS%d:CORR:TYPE%d?' %(int(channel_no), int(trace_no)))
    return cal_type

def open_calibration(channel_no, port_no):
    #Syntax= :SENSe{[1]-4}:CORRection:COLLect[:ACQuire]:OPEN <numeric>
    #Description= This command measures the calibration data of the open standard for the specified port, for the selected channel (Ch).
    inst.write(':SENS%d:CORR:COLl:OPEN %d' %(int(channel_no), int(port_no)))
    state = wait()
    return state

def short_calibration(channel_no, port_no):
    #Syntax= :SENSe{[1]-4}:CORRection:COLLect[:ACQuire]:SHORt <numeric>
    #Description= This command measures the calibration data of the short standard for the specified port, for the selected channel (Ch).
    inst.write(':SENS%d:CORR:COLL:SHOR %d' %(int(channel_no), int(port_no)))
    state = wait()
    return state

def load_calibration(channel_no, port_no):
    #Syntax= :SENSe{[1]-4}:CORRection:COLLect[:ACQuire]:LOAD <numeric>
    #Description= This command measures the calibration data of the load standard for the specified port, for the selected channel (Ch).
    inst.write(':SENS%d:CORR:COLL:LOAD %d' %(int(channel_no), int(port_no)))
    state = wait()
    return state

def through_12_calibration(channel_no, port_no_1, port_no_2):
    #Syntax= :SENSe{[1]-4}:CORRection:COLLect[:ACQuire]:LOAD <numeric>
    #Description= This command measures the calibration data of the load standard for the specified port, for the selected channel (Ch).
    inst.write(':SENS%d:CORR:COLL:THRU %d,%d' %(int(channel_no), int(port_no_1), int(port_no_2)))
    state1 = wait()
    state2 = 0
    if state1 == 1:
        inst.write(':SENS%d:CORR:COLL:THRU %d,%d' %(int(channel_no), int(port_no_2), int(port_no_1)))
        state2 = wait()
    return state1, state2

#to save the calibration
def save_calibration(channel_no):
    inst.write(':SENS%d:CORR:COLL:SAVE' %(int(channel_no)))
    state = wait()
    return state

